# 调用必备库

In [1]:
import PyQt5

In [2]:
%matplotlib qt5

In [3]:
import time
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

np.random.seed(42)
torch.manual_seed(42)

# 先随便搭建个简单的神经网络


$u_{t}-2u_{xx}=0$         

0<=x<=3

$u(t,0)=10,u(3,t)=40$

$u_t(0,x)=25$



## 真实值求解

In [4]:
a=1
l=3
from scipy import integrate
import numpy as np
def u(x,t):#x在前，t在后
    sum=0
    err=0
    for n in range(1,300):  
        sum=sum+1/2/n*np.exp(-8*n**2*np.pi**2*t/9)*np.sin(2*n*np.pi*x/3)
    return 60/np.pi*sum+10*(x+1)

## 神经网络求解

In [5]:
class zsrDGM_net(nn.Module):
    def __init__(self,numl,numn):
        # numl是有多少层隐藏层
        # numn是每层的神经元数量
        super(zsrDGM_net, self).__init__()
        self.input_layer = nn.Linear(2, numn)#前面的数字代表几个输入
        self.hidden_layers = nn.ModuleList([nn.Linear(numn, numn) for i in range(numl)])
        self.output_layer = nn.Linear(numn, 1)
    def forward(self, x):
        o = self.act(self.input_layer(x))
        for i, li in enumerate(self.hidden_layers):
            o = self.act(li(o))        
        out = self.output_layer(o)        
        return out
    def act(self, x):
        return x * torch.tanh(x)

In [6]:
class PDE():
    def __init__(self, net,t):
        self.net=net
        self.t=t
    def sample(self,size=2**12):        
        x = torch.cat((torch.rand([N1*size, 1]) * self.t, torch.full([N1*size, 1], 0) + torch.rand([N1*size, 1]) * 3), dim=1)
        x_init = torch.full([N2*size, 1], 0) + torch.rand([N2*size, 1]) * 3 
        x_initial = torch.cat((torch.zeros(N2*size, 1), x_init), dim=1)
        x_boundary_left = torch.cat((torch.rand([N3*size, 1])*self.t, torch.full([N3*size, 1], 0)), dim=1)
        x_boundary_right = torch.cat((torch.rand([N3*size, 1])*self.t, torch.full([N3*size, 1], 3)), dim=1)
        return x, x_initial, x_init, x_boundary_left, x_boundary_right
    def loss_func(self,size=2**12):
        x_train, x_initial, x_init, x_boundary_left, x_boundary_right = self.sample(size=size)
        x = Variable(x_train, requires_grad=True)
        d = torch.autograd.grad(net(x), x, grad_outputs=torch.ones_like(net(x)), create_graph=True)
        dt = d[0][:, 0].unsqueeze(-1)
        dx = d[0][:, 1].unsqueeze(-1)
        dxx = torch.autograd.grad(dx, x, grad_outputs=torch.ones_like(dx), create_graph=True)[0][:, 1].unsqueeze(-1)
        loss_fn = nn.MSELoss(reduction='mean')
        loss1 = loss_fn(dt, 2*dxx)
        loss2 = loss_fn(net(x_initial), torch.zeros([N2*size,1])+torch.full([N2*size, 1], 25))
        loss3 = loss_fn(net(x_boundary_left), torch.zeros([N3*size,1])+torch.full([N3*size, 1], 10))
        loss4 = loss_fn(net(x_boundary_right),torch.zeros([N3*size,1])+torch.full([N3*size, 1], 40))
        loss = loss1 + loss2 + loss3 + loss4
        #print(loss1,loss2,loss3,loss4)
        return loss
        
        

In [7]:
class Train():
    def __init__(self, net, eq, BATCH_SIZE):
        self.errors = []
        self.BATCH_SIZE = BATCH_SIZE
        self.net = net
        self.model = eq
    def train(self, epoch, lr):
        optimizer = optim.Adam(self.net.parameters(), lr)
        avg_loss = 0
        for e in range(epoch):
            optimizer.zero_grad()
            loss = self.model.loss_func(self.BATCH_SIZE)
            avg_loss = avg_loss + float(loss.item())
            loss.backward()
            optimizer.step()
            if e % 100 == 99:
                loss = avg_loss/100
                print("Epoch {} - lr {} -  loss: {}".format(e, lr, loss))
                avg_loss = 0

                error = self.model.loss_func(2**12)
                self.errors.append(error.detach())
    def get_errors(self):
        return self.errors

In [16]:
net = zsrDGM_net(numl=7, numn=200)
t=0.5
N1=1
N2=1
N3=3
equation = PDE(net,t)
train = Train(net, equation, BATCH_SIZE=2**8)
train.train(epoch=3000, lr=0.001)
torch.save(net, 'test.pkl')
errors = train.get_errors()

Epoch 99 - lr 0.001 -  loss: 512.2448738098144
Epoch 199 - lr 0.001 -  loss: 51.5113646697998
Epoch 299 - lr 0.001 -  loss: 29.10129518508911
Epoch 399 - lr 0.001 -  loss: 21.429500637054442
Epoch 499 - lr 0.001 -  loss: 17.69944037437439
Epoch 599 - lr 0.001 -  loss: 16.778919048309326
Epoch 699 - lr 0.001 -  loss: 15.39359845161438
Epoch 799 - lr 0.001 -  loss: 15.043896923065185
Epoch 899 - lr 0.001 -  loss: 14.81040147781372
Epoch 999 - lr 0.001 -  loss: 15.527311601638793
Epoch 1099 - lr 0.001 -  loss: 13.024066648483277
Epoch 1199 - lr 0.001 -  loss: 11.882826948165894
Epoch 1299 - lr 0.001 -  loss: 11.5314830493927
Epoch 1399 - lr 0.001 -  loss: 11.713484249114991
Epoch 1499 - lr 0.001 -  loss: 10.512985763549805
Epoch 1599 - lr 0.001 -  loss: 10.459495143890381
Epoch 1699 - lr 0.001 -  loss: 11.571468787193298
Epoch 1799 - lr 0.001 -  loss: 9.496496448516845
Epoch 1899 - lr 0.001 -  loss: 9.653167033195496
Epoch 1999 - lr 0.001 -  loss: 8.437394328117371
Epoch 2099 - lr 0.001 -

In [9]:
fig = plt.figure()
plt.plot(np.log(errors), '-b', label='Errors')
plt.title('Training Loss', fontsize=10)
plt.savefig('err.jpg')
plt.close(fig)

In [8]:
u_true=[u(test1/10,0.1) for test1 in range(0,30)]
net=torch.load('test.pkl')
x_1 = torch.tensor([[i/10] for i in range(0,30)])
x_11 = torch.cat((torch.full([30, 1],0.1), x_1), dim=1)
ysolve=net(x_11)
ysolve=ysolve.detach().numpy()
x_range=[i/10 for i in range(0,30)]
fig=plt.figure(dpi=60)
plt.plot(x_range,u_true)
plt.plot(x_range,(ysolve))
plt.show()

In [10]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
net=torch.load('test.pkl')
fig, ax = plt.subplots()
err=[]
# 定义存储数据的列表
xdata = []
ydata = []
ydata2= []
# 接收line2D对象
line, = plt.plot(xdata, ydata, 'b')
line1, =plt.plot(xdata,ydata2,'g')
xdata=[i/100 for i in range(0,300)]
# 定义更新函数
def update(frames):
    x_1 = torch.tensor([[i/100] for i in range(0,300)])
    x_11 = torch.cat((torch.full([300, 1],frames), x_1), dim=1)
    ysolve1=net(x_11)
    ysolve=ysolve1.detach().numpy()
    ydata=[ysolve]
    line.set_data(xdata, ydata)
    u_true=[u(test1/100,frames) for test1 in range(0,300)]
    ydata2=u_true
    #print(ydata2)
    line1.set_data(xdata,ydata2)
    return line,line1


def init_figure():
    ax.set_xlim(0, 3)
    ax.set_ylim(10,40)
    ax.set_xlabel('x')
    ax.set_ylabel('u')


# 调用生成动画的函数生成动图
ani = animation.FuncAnimation(
    fig=fig,
    func=update,
    frames=np.linspace(0, 0.5, 100),    # [1, 2, 3]
    init_func=init_figure,
    interval=100,   #  每隔多少时间生成一帧图像，单位是ms
    repeat=True,   # 设置不重复，但是还是重复较好
)

#plt.show()   # 如果要保存视频和gif就不要show()
#ani.save('shuli.gif', writer='pillow')
ani.save('1.mp4', writer='ffmpeg')  # 注意，pillow现在似乎不能报错为mp4格式了，可以使用ffmpeg